```
:시험관: 미션: “여기어때 숙소 데이터 수집·적재·요약”
목표 : 국내 숙소 플랫폼의 공개 웹페이지를 가볍게 탐색해 숙소 메타데이터를 수집하고, 이를 MongoDB에 저장한 뒤 간단한 통계 리포트를 산출
범위
사이트: https://www.yeogi.com/
도시 1곳을 선정(예: 서울)하고, 리스트 페이지 상위 2–3페이지 목표로 수집.
수집 항목 (가능한 한 확보 : 자유)
url(고유), name(숙소명), location(구/동 등 텍스트),
price_min(최저가, 숫자), rating(평점, 숫자), review_count(리뷰 수, 숫자)
아래 데이터를 Aggregation으로 산출.
평점 상위 10: rating 내림차순, 동률이면 review_count 많은 순
```

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

service = Service(ChromeDriverManager().install())
options = Options()

options.add_argument("--disable-dev-shm-usage")
options.add_argument("--start-maximaized")
options.add_argument("--window-size=1920x1080")
options.add_argument("--lang=ko_KR")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service = service, options = options)

data = []

for i in range(2) :
    if i == 0 :
        url = "https://www.yeogi.com/domestic-accommodations?keyword=%EA%B2%BD%EC%A3%BC&checkIn=2025-09-11&checkOut=2025-09-12&personal=2&freeForm=false"
    else:
        url = f"https://www.yeogi.com/domestic-accommodations?keyword=%EA%B2%BD%EC%A3%BC&checkIn=2025-09-11&checkOut=2025-09-12&personal=2&freeForm=false&page={i+1}"

    driver.get(url)
    time.sleep(3)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    links = driver.find_elements(By.CSS_SELECTOR, "a.gc-thumbnail-type-seller-card.css-vbesvf")
    for link in links :
        l = link.get_attribute("href")
        driver.get(l)
        time.sleep(3)

        name = driver.find_element(By.XPATH, "//section[@id='overview']//h1[@class='css-ey2zdd']")
        url = link
        location = driver.find_element(By.CSS_SELECTOR, "div.css-y3ur5y p.css-ix9gtu").text
        try: 
            price_min = driver.find_element(By.CSS_SELECTOR, "div.stay-price-tag.css-93efn5 span.css-12yvfi0").text
        except:
            price_min = "None"
        try:
            rating = driver.find_element(By.CSS_SELECTOR, "div.css-y3ur5y span.css-gvatwm").text
        except:
            rating = "None"
        try:
            review_count = driver.find_element(By.CSS_SELECTOR, "div.css-y3ur5y span.css-1k0ukhp").text
        except:
            review_count = "None"

        data.append({"name": name, "url": url, "location": location, "price_min": price_min, "rating": rating, "review_count": review_count})

print(data)

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=139.0.7258.157); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x66a8a3+63283]
	GetHandleVerifier [0x0x66a8e4+63348]
	(No symbol) [0x0x4a3e43]
	(No symbol) [0x0x493790]
	(No symbol) [0x0x4b1602]
	(No symbol) [0x0x517cdc]
	(No symbol) [0x0x531ff9]
	(No symbol) [0x0x511216]
	(No symbol) [0x0x4e0855]
	(No symbol) [0x0x4e16f4]
	GetHandleVerifier [0x0x8dbb43+2623955]
	GetHandleVerifier [0x0x8d6daa+2604090]
	GetHandleVerifier [0x0x69069a+218410]
	GetHandleVerifier [0x0x680ed8+154984]
	GetHandleVerifier [0x0x68742d+180925]
	GetHandleVerifier [0x0x6722b8+94536]
	GetHandleVerifier [0x0x672442+94930]
	GetHandleVerifier [0x0x65d5ea+9338]
	BaseThreadInitThunk [0x0x76dbfcc9+25]
	RtlGetAppC

In [ ]:
# MongoDB 삽입
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["Mission"]
collection = db.yeogi

result = collection.insert_many(data)


##### 여기어때 사이트에서 차단되어서 실행 결과는 확인하지 못했습니다😢